 # Schools_SynthPops
 This notebook is intended to prepare the school data needed by the `generate_synthetic_population` function of [Synthpops](https://docs.idmod.org/projects/synthpops/en/latest/index.html)
 
 ## Useful Links
 ### Data and data interpreation
 [Open Data](https://dati.istruzione.it/opendata/) <br>
 [Codici Meccanografici](http://www.comuni-italiani.it/001/scuole/)<br>
 ### History of italian provinces
 [Sud Sardegna](https://it.wikipedia.org/wiki/Provincia_del_Sud_Sardegna)<br>
 [Olbia-Tempio](https://it.wikipedia.org/wiki/Provincia_di_Olbia-Tempio)<br>
 [Ogliastra](https://it.wikipedia.org/wiki/Ogliastra)<br>
 
 ## Ideas
 
 - With respect to ICs, which are school (infancy, elementary ad Lower) complexes whose nature (exact mechanographic code) is impossible to determine:
 > direi allora facciamo una bella media dei volumi di tutti gli asili, elementari e medie e poi dividiamo il volume degli IC per questo valore. Così otteniamo il numero di "scuole efficaci" a cui un IC corrisponde

In [1]:
import pandas as pd
import numpy as np
import ast
import copy

In [2]:
#pupils_per_class = 21.6 # see D:\IlMIoDrive\magistrale\2anno\primo_periodo\epi\Data\CSV\2020\Infrastructural\DCIS_SCUOLE_15102020182434230.csv (taken from http://dati.istat.it/#)

In [3]:
#class_volume = pupils_per_class*((4))*4 #number of students*average surface area per student*

In [4]:
code_province_df = pd.read_csv(r"D:\IlMIoDrive\magistrale\2anno\primo_periodo\epi\Data\CSV\2020\Administrative\ItalianProvinces.csv")
code_province_df["abbreviation"] = ["NA" if pd.isnull(abbr) else abbr for abbr in code_province_df["abbreviation"]]
code_province_df.head()

,code,name,population,abbreviation
0,1,Torino,2252379,TO
1,2,Vercelli,170296,VC
2,3,Novara,368040,NO
3,4,Cuneo,586568,CN
4,5,Asti,213216,AT


In [5]:
ORD_MEC_df = pd.read_csv(r"D:\IlMIoDrive\magistrale\2anno\primo_periodo\epi\Data\CSV\2020\Infrastructural\ORDINESCOL2_MEC.tsv", sep = "\t")
ORD_MEC_df["MEC"] = [ast.literal_eval(string) for string in ORD_MEC_df["MEC"]]
ORD_MEC_df

,ORDINESCOL2,MEC
0,PRI,[EE]
1,PRE,[AA]
2,LOW,[MM]
3,UPP,"[PQ, PM, RC, RH, RI, RS, RH, RA , TD, SD, TA, ..."


In [6]:
def dct_to_kv_pairs(dct):
    return [(list(dct.keys())[i], list(dct.values())[i]) for i in range(len(dct))]

In [7]:
ORD_MEC_dct = {row[0]:row[1] for i,row in ORD_MEC_df.iterrows()}
print(ORD_MEC_dct)
ORD_MEC_lst = dct_to_kv_pairs(ORD_MEC_dct)
print(dct_to_kv_pairs(ORD_MEC_dct))

{'PRI': ['EE'], 'PRE': ['AA'], 'LOW': ['MM'], 'UPP': ['PQ', 'PM', 'RC', 'RH', 'RI', 'RS', 'RH', 'RA ', 'TD', 'SD', 'TA', 'TB', 'TF', 'TL', 'TN ', 'SL', 'PC', 'PS', 'PL', 'TE', 'PM']}
[('PRI', ['EE']), ('PRE', ['AA']), ('LOW', ['MM']), ('UPP', ['PQ', 'PM', 'RC', 'RH', 'RI', 'RS', 'RH', 'RA ', 'TD', 'SD', 'TA', 'TB', 'TF', 'TL', 'TN ', 'SL', 'PC', 'PS', 'PL', 'TE', 'PM'])]


In [8]:
scuole_df = pd.read_csv(r"D:\IlMIoDrive\magistrale\2anno\primo_periodo\epi\Data\CSV\2020\Infrastructural\EDICONSISTENZASTA20181920180901.csv")
scuole_df = scuole_df[scuole_df["VOLUME"] != "-"]
scuole_df["VOLUME"] = [float(volume) for volume in scuole_df["VOLUME"].tolist()] # add column "CAPIENZA_SCUOLA"
scuole_df["school_type"] = [string[2:4] for string in scuole_df["CODICESCUOLA"]] 
scuole_df.drop(["CODICEEDIFICIO","SPERFICIEAREATOTALE","SPERFICIEAREALIBERA"], axis = 1, inplace = True) #
print(scuole_df.head())
ORDINESCOL2 = []
for Type in scuole_df["school_type"].tolist():
    found = False
    if Type == "IC":
        ORDINESCOL2.append("IC")
    else:
        for tup in ORD_MEC_lst:
            if Type in tup[1]:
                ORDINESCOL2.append(tup[0])
                found = True
                break
        if not found:
            ORDINESCOL2.append("UPP")
        
scuole_df["ORDINESCOL2"] = ORDINESCOL2

scuole_df["abbreviation"]  = [string[:2] for string in scuole_df["CODICESCUOLA"]]
scuole_df["abbreviation"] = ["FC" if province == "FO" else province for province in scuole_df["abbreviation"] ]
scuole_df["abbreviation"] = ["PU" if province == "PS" else province for province in scuole_df["abbreviation"] ]
scuole_df.drop(["school_type","ANNOSCOLASTICO"], axis = 1, inplace = True)
cols = scuole_df.columns.tolist()
scuole_df = scuole_df[[cols[-1]]+[cols[0]]+[cols[-2]]+[cols[-3]]]
scuole_df.head(20)

   ANNOSCOLASTICO CODICESCUOLA   VOLUME school_type
0          201819   FGEE82302X   7000.0          EE
1          201819   COTD00101T  35000.0          TD
2          201819   CTAA08305Q    435.0          AA
3          201819   MITF11000E   9068.0          TF
5          201819   CSIS06100T  18600.0          IS


,abbreviation,CODICESCUOLA,ORDINESCOL2,VOLUME
0,FG,FGEE82302X,PRI,7000.00
1,CO,COTD00101T,UPP,35000.00
2,CT,CTAA08305Q,PRE,435.00
3,MI,MITF11000E,UPP,9068.00
5,CS,CSIS06100T,UPP,18600.00
6,AL,ALEE818052,PRI,3360.00
7,MO,MORC004013,UPP,40522.35
8,CL,CLAA82002B,PRE,1827.49
9,PC,PCAA819033,PRE,1650.00
10,VA,VAEE873015,PRI,18950.00


In [9]:
ICs_df = copy.deepcopy(scuole_df[scuole_df["ORDINESCOL2"] == "IC"])
ICs_df["count"] = [1 for i in range(len(ICs_df))]
ICs_df = ICs_df.groupby(["abbreviation"],as_index = False).agg({"count":"sum"})
ICs_df

,abbreviation,count
0,AG,46
1,AN,45
2,AO,15
3,AP,30
4,AQ,25
...,...,...
90,VE,66
91,VI,80
92,VR,66
93,VT,30


In [10]:
ICs_per_province_dct = {row[0]:row[1] for i,row in ICs_df.iterrows() }
ICs_per_province_dct

{'AG': 46,
 'AN': 45,
 'AO': 15,
 'AP': 30,
 'AQ': 25,
 'AR': 32,
 'AV': 44,
 'BA': 95,
 'BG': 102,
 'BL': 29,
 'BN': 35,
 'BO': 74,
 'BR': 30,
 'BS': 104,
 'CA': 70,
 'CB': 22,
 'CE': 98,
 'CH': 36,
 'CL': 23,
 'CO': 48,
 'CR': 28,
 'CS': 94,
 'CT': 118,
 'CZ': 41,
 'EN': 19,
 'FC': 24,
 'FE': 26,
 'FG': 61,
 'FI': 65,
 'FR': 50,
 'GE': 60,
 'GO': 1,
 'GR': 21,
 'IM': 18,
 'IS': 11,
 'KR': 27,
 'LC': 29,
 'LE': 76,
 'LI': 11,
 'LO': 18,
 'LT': 58,
 'LU': 37,
 'MC': 28,
 'ME': 69,
 'MI': 305,
 'MN': 35,
 'MO': 48,
 'MS': 17,
 'MT': 25,
 'NA': 194,
 'NO': 1,
 'NU': 31,
 'OR': 16,
 'PA': 118,
 'PC': 14,
 'PD': 77,
 'PE': 30,
 'PG': 38,
 'PI': 33,
 'PN': 4,
 'PO': 19,
 'PR': 32,
 'PT': 24,
 'PU': 35,
 'PV': 36,
 'PZ': 48,
 'RA': 30,
 'RC': 59,
 'RE': 46,
 'RG': 29,
 'RI': 22,
 'RM': 331,
 'RN': 16,
 'RO': 30,
 'SA': 90,
 'SI': 24,
 'SO': 22,
 'SP': 16,
 'SR': 52,
 'SS': 44,
 'SV': 18,
 'TA': 56,
 'TE': 28,
 'TO': 1,
 'TP': 36,
 'TR': 16,
 'TS': 2,
 'TV': 66,
 'UD': 1,
 'VA': 76,
 'VE': 66

In [11]:
for abbr in code_province_df["abbreviation"]:
    try:
        ICs_per_province_dct[abbr]
    except KeyError as e:
        ICs_per_province_dct[abbr] = 1

In [12]:
scuole_gb_provincia_ordine_df = scuole_df.groupby(["abbreviation","ORDINESCOL2"], as_index = False).agg({"VOLUME":"sum"})
scuole_gb_provincia_ordine_df.head()

,abbreviation,ORDINESCOL2,VOLUME
0,AG,IC,526157.78
1,AG,LOW,860769.90
2,AG,PRE,855734.41
3,AG,PRI,1284800.12
4,AG,UPP,2077724.38


In [13]:
def encapsulate(value):
    return [value]

In [14]:
def my_reduce(lst):
    return [lst[0]+lst[1],lst[2]+lst[3],(lst[4]+lst[5])/2.,lst[6]+lst[7]]

In [15]:
students_by_province_df = pd.read_csv(r"D:\IlMIoDrive\magistrale\2anno\primo_periodo\epi\Data\CSV\2020\Infrastructural\DCIS_SCUOLE_iscritti_per_scuola_2.csv")
print(np.unique(students_by_province_df["ORDINESCOL2"].to_list()))
print(np.unique(students_by_province_df["NATGIUR1"].to_list()))
print(len(np.unique(students_by_province_df["Territorio"].to_list())))
print(len(np.unique(students_by_province_df["Seleziona anno (anno t= anno sc. t-1 / t)"].to_list())))
#print(np.unique(students_by_province_df["SEXISTAT1"].to_list()))
print(np.unique(students_by_province_df["TIME"].to_list()))
print(np.unique(students_by_province_df["Gestione della scuola"].to_list()))
students_by_province_df = students_by_province_df.drop(["Flag Codes","Flags", "Seleziona anno (anno t= anno sc. t-1 / t)","TIPO_DATO_ISTR","Ordine scolastico","NATGIUR1","TIME","Gestione della scuola","ITTER107"], axis =1)
#students_by_province_df.sort_values(by = ["Territorio","ORDINESCOL2","Tipo dato"], inplace = True)

sud_sardegna_pre = students_by_province_df[students_by_province_df["Territorio"] == 'Medio Campidano']
students_by_province_df["Territorio"] = ["Forli'-Cesena" if province == 'Forlì-Cesena' else province  for province in students_by_province_df["Territorio"].to_list() ]
students_by_province_df["Territorio"] = ['Massa Carrara' if province == 'Massa-Carrara' else province  for province in students_by_province_df["Territorio"].to_list() ]
students_by_province_df["Territorio"] = ['Sud Sardegna' if province in ['Carbonia-Iglesias','Medio Campidano'] else province  for province in students_by_province_df["Territorio"].to_list() ]
students_by_province_df["Territorio"] = ['Sassari' if province == 'Olbia-Tempio' else province  for province in students_by_province_df["Territorio"].to_list() ]
students_by_province_df["Territorio"] = ['Nuoro' if province == 'Ogliastra' else province  for province in students_by_province_df["Territorio"].to_list() ]
students_by_province_df["Territorio"] = ['Trento' if province == 'Provincia Autonoma Trento' else province  for province in students_by_province_df["Territorio"].to_list() ]
students_by_province_df["Territorio"] = ['Bolzano' if province == 'Provincia Autonoma Bolzano / Bozen' else province  for province in students_by_province_df["Territorio"].to_list() ]
students_by_province_df["Territorio"] = ['Aosta' if province == "Valle d'Aosta / Vallée d'Aoste" else province  for province in students_by_province_df["Territorio"].to_list() ]
students_by_province_df["abbreviation"] = [couple[1] for Territorio in students_by_province_df["Territorio"].to_list() for couple in zip(code_province_df["name"].to_list(),code_province_df["abbreviation"].to_list() ) if Territorio == couple[0]]
students_by_province_df["Value"] = [encapsulate(value) for value in students_by_province_df["Value"] ] 
students_by_province_df.sort_values(by = ["abbreviation","ORDINESCOL2","Tipo dato"], inplace = True)
milano = students_by_province_df[students_by_province_df["abbreviation"] == "MI"]
sud_sardegna = students_by_province_df[students_by_province_df["Territorio"] == "Sud Sardegna"]
#students_by_province_df.groupby(["Territorio","Tipo dato","ORDINESCOL2"])
#students_by_province_df.head(20)
students_by_province_gb_df = students_by_province_df.groupby(["Territorio","abbreviation","ORDINESCOL2"], as_index = False).agg({"Value":"sum"})
students_by_province_gb_df.columns = ["Territorio","abbreviation","ORDINESCOL2","c_i_ic_s"]
sud_sardegna_post = students_by_province_gb_df[students_by_province_gb_df["Territorio"] == "Sud Sardegna"]
students_by_province_gb_df["c_i_ic_s"] = [c_i_ic_s if len(c_i_ic_s) == 4 else my_reduce(c_i_ic_s) for c_i_ic_s in students_by_province_gb_df["c_i_ic_s"]]
sud_sardegna_post2 = students_by_province_gb_df[students_by_province_gb_df["Territorio"] == "Sud Sardegna"]
students_by_province_gb_df.head()


['LOW' 'PRE' 'PRI' 'UPP']
['PUB']
110
1
[2018]
['pubblica']


,Territorio,abbreviation,ORDINESCOL2,c_i_ic_s
0,Agrigento,AG,LOW,"[668.0, 13557.0, 20.3, 63.0]"
1,Agrigento,AG,PRE,"[486.0, 9631.0, 19.8, 131.0]"
2,Agrigento,AG,PRI,"[1066.0, 19479.0, 18.3, 120.0]"
3,Agrigento,AG,UPP,"[1025.0, 21478.0, 21.0, 55.0]"
4,Alessandria,AL,LOW,"[483.0, 9816.0, 20.3, 58.0]"


In [16]:
#milano

In [17]:
#sud_sardegna

In [18]:
#sud_sardegna_post2

In [19]:
#set(students_by_province_gb_df["ORDINESCOL2"])

In order to estimate ICs' capacity, we need to average over school orders (**ORDINESCOL2**) of **LOW**, **PRE** and **PRI** departments in every province.  

In [20]:
students_by_province_gb_noUPP_df = copy.deepcopy(students_by_province_gb_df[students_by_province_gb_df["ORDINESCOL2"] != "UPP"])
students_by_province_gb_noUPP_df.head(10)

,Territorio,abbreviation,ORDINESCOL2,c_i_ic_s
0,Agrigento,AG,LOW,"[668.0, 13557.0, 20.3, 63.0]"
1,Agrigento,AG,PRE,"[486.0, 9631.0, 19.8, 131.0]"
2,Agrigento,AG,PRI,"[1066.0, 19479.0, 18.3, 120.0]"
4,Alessandria,AL,LOW,"[483.0, 9816.0, 20.3, 58.0]"
5,Alessandria,AL,PRE,"[329.0, 7377.0, 22.4, 133.0]"
6,Alessandria,AL,PRI,"[947.0, 15730.0, 16.6, 141.0]"
8,Ancona,AN,LOW,"[574.0, 12831.0, 22.4, 62.0]"
9,Ancona,AN,PRE,"[469.0, 10668.0, 22.7, 147.0]"
10,Ancona,AN,PRI,"[1100.0, 21368.0, 19.4, 117.0]"
12,Aosta,AO,LOW,"[171.0, 3445.0, 20.1, 20.0]"


In [21]:
#students_by_province_gb_noUPP_df["c_i_ic_s"][0]

In [22]:
#len(students_by_province_gb_noUPP_df)

In [23]:
#milano.head()

In [24]:
 #for ICs_per_province_dct["AL"]

In [25]:
# this function takes as inputs the three lists (grouped in a series) containing the c_i_ic_s of the LOW, PRI and PRE school orders of a province and produces the list of the estimated s_c_i_ic of the ICs of that province (sums the first three elements and evaluates the fourth dividing the third by the second)
def sum_average(series):
    #print(series)
    zipped = list(zip(series.tolist()[0],series.tolist()[1],series.tolist()[2]))
    return [(sum(zipped[0])/sum(zipped[3]))*ICs_per_province_dct[zipped[4][0]],(sum(zipped[1])/sum(zipped[3]))*ICs_per_province_dct[zipped[4][0]],((sum(zipped[1])/sum(zipped[3]))*ICs_per_province_dct[zipped[4][0]])/((sum(zipped[0])/sum(zipped[3]))*ICs_per_province_dct[zipped[4][0]]),sum(zipped[3])]

In [26]:
students_by_province_gb_noUPP_df["c_i_ic_s"] = [row[3] + [row[1]] for i,row in students_by_province_gb_noUPP_df.iterrows()]
#students_by_province_gb_noUPP_df.head(20)
students_by_province_gb_noUPP_gb_df = students_by_province_gb_noUPP_df.groupby(["Territorio","abbreviation"],as_index = False).agg({"c_i_ic_s":sum_average})
students_by_province_gb_noUPP_gb_df.head()

,Territorio,abbreviation,c_i_ic_s
0,Agrigento,AG,"[325.2229299363057, 6250.579617834394, 19.2193..."
1,Alessandria,AL,"[5.298192771084337, 99.16566265060241, 18.7168..."
2,Ancona,AN,"[295.8128834355828, 6193.29754601227, 20.93653..."
3,Aosta,AO,"[57.704918032786885, 954.7540983606557, 16.545..."
4,Arezzo,AR,"[185.15175097276264, 3730.5525291828794, 20.14..."


In [27]:
students_by_province_gb_noUPP_gb_df["ORDINESCOL2"] = ["IC" for i in range(len(students_by_province_gb_noUPP_gb_df))]
students_by_province_gb_noUPP_gb_df.head()

,Territorio,abbreviation,c_i_ic_s,ORDINESCOL2
0,Agrigento,AG,"[325.2229299363057, 6250.579617834394, 19.2193...",IC
1,Alessandria,AL,"[5.298192771084337, 99.16566265060241, 18.7168...",IC
2,Ancona,AN,"[295.8128834355828, 6193.29754601227, 20.93653...",IC
3,Aosta,AO,"[57.704918032786885, 954.7540983606557, 16.545...",IC
4,Arezzo,AR,"[185.15175097276264, 3730.5525291828794, 20.14...",IC


In [28]:
students_by_province_gb_df = pd.concat([students_by_province_gb_df, students_by_province_gb_noUPP_gb_df], ignore_index=True)
students_by_province_gb_df

,Territorio,abbreviation,ORDINESCOL2,c_i_ic_s
0,Agrigento,AG,LOW,"[668.0, 13557.0, 20.3, 63.0]"
1,Agrigento,AG,PRE,"[486.0, 9631.0, 19.8, 131.0]"
2,Agrigento,AG,PRI,"[1066.0, 19479.0, 18.3, 120.0]"
3,Agrigento,AG,UPP,"[1025.0, 21478.0, 21.0, 55.0]"
4,Alessandria,AL,LOW,"[483.0, 9816.0, 20.3, 58.0]"
...,...,...,...,...
530,Vercelli,VC,IC,"[5.297297297297297, 95.3445945945946, 17.99872..."
531,Verona,VR,IC,"[518.9813664596273, 10532.944099378883, 20.295..."
532,Vibo Valentia,VV,IC,"[90.12280701754386, 1459.0438596491229, 16.189..."
533,Vicenza,VI,IC,"[615.9834368530021, 12496.23188405797, 20.2866..."


In [29]:
students_by_province_gb_df.sort_values(by = ["abbreviation","ORDINESCOL2"], inplace = True)
students_by_province_gb_df

,Territorio,abbreviation,ORDINESCOL2,c_i_ic_s
428,Agrigento,AG,IC,"[325.2229299363057, 6250.579617834394, 19.2193..."
0,Agrigento,AG,LOW,"[668.0, 13557.0, 20.3, 63.0]"
1,Agrigento,AG,PRE,"[486.0, 9631.0, 19.8, 131.0]"
2,Agrigento,AG,PRI,"[1066.0, 19479.0, 18.3, 120.0]"
3,Agrigento,AG,UPP,"[1025.0, 21478.0, 21.0, 55.0]"
...,...,...,...,...
532,Vibo Valentia,VV,IC,"[90.12280701754386, 1459.0438596491229, 16.189..."
416,Vibo Valentia,VV,LOW,"[275.0, 4872.0, 17.7, 53.0]"
417,Vibo Valentia,VV,PRE,"[171.0, 3149.0, 18.4, 91.0]"
418,Vibo Valentia,VV,PRI,"[488.0, 7100.0, 14.5, 84.0]"


$numero_classi_scuola_provincia_ordine_volume = (numero_classi_provicia_ordine/somma_volumi_scuole_provincia_ordine)*volume$ <br>
$studenti_scuola_provincia_ordine_volume = numero_classi_scuola_provincia_ordine_volume *iscritti_per_classe_provincia_ordine$<br>

In [30]:
# students_by_province_df.set_index("Territorio",inplace = True)
# print(students_by_province_df.head())
# students_by_province_t_df = students_by_province_df.transpose()
# students_by_province_t_df.head()

In [31]:
# torino  = students_by_province_gb_df[students_by_province_gb_df["Territorio"] == "Torino"]
# torino

In [32]:
# province_set = set(co)
# provincia_ordine_sommaVolumi_dct = {}

In [33]:
#sum([1 if sal == "-" else 0 for sal in scuole_df["SPERFICIEAREALIBERA"]])

In [34]:
#sum([1 if sal > sat else 0 for sal,sat in zip(scuole_df["SPERFICIEAREALIBERA"],scuole_df["SPERFICIEAREATOTALE"])])

In [35]:
len(scuole_df)

56568

In [36]:
print(set(scuole_df["abbreviation"]) - set(code_province_df["abbreviation"]))
print(set(code_province_df["abbreviation"])-set(scuole_df["abbreviation"]))

set()
{'MB', 'TN', 'FM', 'BT', 'SU', 'BZ'}


In [37]:
scuole_gb_provincia_ordine_merged_df = scuole_gb_provincia_ordine_df.merge(right =students_by_province_gb_df , on = ["abbreviation","ORDINESCOL2"],validate = "one_to_one")
scuole_gb_provincia_ordine_merged_df

,abbreviation,ORDINESCOL2,VOLUME,Territorio,c_i_ic_s
0,AG,IC,526157.78,Agrigento,"[325.2229299363057, 6250.579617834394, 19.2193..."
1,AG,LOW,860769.90,Agrigento,"[668.0, 13557.0, 20.3, 63.0]"
2,AG,PRE,855734.41,Agrigento,"[486.0, 9631.0, 19.8, 131.0]"
3,AG,PRI,1284800.12,Agrigento,"[1066.0, 19479.0, 18.3, 120.0]"
4,AG,UPP,2077724.38,Agrigento,"[1025.0, 21478.0, 21.0, 55.0]"
...,...,...,...,...,...
494,VV,IC,192587.05,Vibo Valentia,"[90.12280701754386, 1459.0438596491229, 16.189..."
495,VV,LOW,379195.50,Vibo Valentia,"[275.0, 4872.0, 17.7, 53.0]"
496,VV,PRE,302966.91,Vibo Valentia,"[171.0, 3149.0, 18.4, 91.0]"
497,VV,PRI,463503.48,Vibo Valentia,"[488.0, 7100.0, 14.5, 84.0]"


In [38]:
#raw_df["CAPIENZA_SCUOLA"] = (raw_df["VOLUME"]/class_volume)* pupils_per_class

In [39]:
scuole_gb_provincia_ordine_merged_df["#classi/volume"] = [row[4][0]/row[2] for i,row in scuole_gb_provincia_ordine_merged_df.iterrows() ]
scuole_gb_provincia_ordine_merged_df

,abbreviation,ORDINESCOL2,VOLUME,Territorio,c_i_ic_s,#classi/volume
0,AG,IC,526157.78,Agrigento,"[325.2229299363057, 6250.579617834394, 19.2193...",0.000618
1,AG,LOW,860769.90,Agrigento,"[668.0, 13557.0, 20.3, 63.0]",0.000776
2,AG,PRE,855734.41,Agrigento,"[486.0, 9631.0, 19.8, 131.0]",0.000568
3,AG,PRI,1284800.12,Agrigento,"[1066.0, 19479.0, 18.3, 120.0]",0.000830
4,AG,UPP,2077724.38,Agrigento,"[1025.0, 21478.0, 21.0, 55.0]",0.000493
...,...,...,...,...,...,...
494,VV,IC,192587.05,Vibo Valentia,"[90.12280701754386, 1459.0438596491229, 16.189...",0.000468
495,VV,LOW,379195.50,Vibo Valentia,"[275.0, 4872.0, 17.7, 53.0]",0.000725
496,VV,PRE,302966.91,Vibo Valentia,"[171.0, 3149.0, 18.4, 91.0]",0.000564
497,VV,PRI,463503.48,Vibo Valentia,"[488.0, 7100.0, 14.5, 84.0]",0.001053


In [40]:
sassari = scuole_gb_provincia_ordine_merged_df[scuole_gb_provincia_ordine_merged_df["abbreviation"] == "TO"]
sassari

,abbreviation,ORDINESCOL2,VOLUME,Territorio,c_i_ic_s,#classi/volume
431,TO,IC,3098.0,Torino,"[7.330357142857143, 153.08603896103895, 20.883...",0.002366
432,TO,LOW,3499354.0,Torino,"[2710.0, 57613.0, 21.3, 204.0]",0.000774
433,TO,PRE,2895872.0,Torino,"[1757.0, 39183.0, 22.3, 519.0]",0.000607
434,TO,PRI,5749262.0,Torino,"[4564.0, 91806.0, 20.1, 509.0]",0.000794
435,TO,UPP,8227808.0,Torino,"[4022.0, 87756.0, 21.8, 155.0]",0.000489


In [41]:
%%time
# scuole_gb_provincia_ordine_merged_df.loc[(scuole_gb_provincia_ordine_merged_df["abbreviation"] == "TO" ) & (scuole_gb_provincia_ordine_merged_df["ORDINESCOL2"] == "PRI" ),"c_i_ic_s" ].tolist()[0]

    
scuole_df["#classi/volume"] = [scuole_gb_provincia_ordine_merged_df.loc[(scuole_gb_provincia_ordine_merged_df["abbreviation"] == row[0] ) & (scuole_gb_provincia_ordine_merged_df["ORDINESCOL2"] == row[2] ),"#classi/volume" ].tolist()[0] for i,row in scuole_df.iterrows()]
scuole_df["iscritti_per_classe"] = [scuole_gb_provincia_ordine_merged_df.loc[(scuole_gb_provincia_ordine_merged_df["abbreviation"] == row[0] ) & (scuole_gb_provincia_ordine_merged_df["ORDINESCOL2"] == row[2] ),"c_i_ic_s" ].tolist()[0][2] for i,row in scuole_df.iterrows()]

Wall time: 55.4 s


In [42]:
scuole_df

,abbreviation,CODICESCUOLA,ORDINESCOL2,VOLUME,#classi/volume,iscritti_per_classe
0,FG,FGEE82302X,PRI,7000.0,0.000408,19.300000
1,CO,COTD00101T,UPP,35000.0,0.000396,21.800000
2,CT,CTAA08305Q,PRE,435.0,0.000357,20.800000
3,MI,MITF11000E,UPP,9068.0,0.000280,21.900000
5,CS,CSIS06100T,UPP,18600.0,0.000530,18.900000
...,...,...,...,...,...,...
58840,VE,VEIC838006,IC,36629.0,0.000575,20.926911
58841,BO,BOEE825026,PRI,7334.0,0.000913,21.300000
58842,VI,VIEE81202C,PRI,415.0,0.000910,19.200000
58843,TO,TOAA029022,PRE,4640.0,0.000607,22.300000


In [43]:
scuole_df["enrollments"] = scuole_df["#classi/volume"]*scuole_df["VOLUME"]*scuole_df["iscritti_per_classe"]

In [44]:
scuole_df.head(50)

,abbreviation,CODICESCUOLA,ORDINESCOL2,VOLUME,#classi/volume,iscritti_per_classe,enrollments
0,FG,FGEE82302X,PRI,7000.00,0.000408,19.300000,55.054738
1,CO,COTD00101T,UPP,35000.00,0.000396,21.800000,302.399635
2,CT,CTAA08305Q,PRE,435.00,0.000357,20.800000,3.227159
3,MI,MITF11000E,UPP,9068.00,0.000280,21.900000,55.579344
5,CS,CSIS06100T,UPP,18600.00,0.000530,18.900000,186.401684
6,AL,ALEE818052,PRI,3360.00,0.000472,16.600000,26.332558
7,MO,MORC004013,UPP,40522.35,0.000278,22.600000,254.266226
8,CL,CLAA82002B,PRE,1827.49,0.000341,20.800000,12.970480
9,PC,PCAA819033,PRE,1650.00,0.000464,23.300000,17.842051
10,VA,VAEE873015,PRI,18950.00,0.000812,21.300000,327.723862


In [45]:
sum(scuole_df["enrollments"].to_list()) #merda!!

8224763.114917536

In [46]:
scuole_df.sort_values(by = ["enrollments"], inplace = True,ascending = False)
scuole_df.head(50)

,abbreviation,CODICESCUOLA,ORDINESCOL2,VOLUME,#classi/volume,iscritti_per_classe,enrollments
19922,PA,PATD02901X,UPP,1971814.0,0.000460,20.600000,18694.903549
7324,NA,NAEE18500X,PRI,1474766.0,0.000628,19.100000,17701.128081
6045,NA,NAEE185011,PRI,1474766.0,0.000628,19.100000,17701.128081
19948,CE,CEPM02000V,UPP,821712.0,0.000964,21.000000,16631.153496
31844,FI,FIEE85301B,PRI,1313121.9,0.000555,21.800000,15880.546479
38088,MN,MNEE80701R,PRI,2472045.0,0.000265,19.500000,12786.589698
4111,FI,FIAA853016,PRE,1313121.9,0.000337,23.600000,10437.639139
57439,CT,CTEE822018,PRI,1139652.0,0.000450,19.500000,10007.450099
42474,FG,FGEE819039,PRI,1237374.0,0.000408,19.300000,9731.900160
28806,TA,TAMM81101X,LOW,1058488.2,0.000411,21.000000,9126.319825


In [47]:
max(scuole_df["enrollments"].to_list())

18694.903549170274

In [48]:
sort(scuole_df["enrollments"].to_list())

NameError: name 'sort' is not defined